In [7]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

# Custom PeepholeLSTM layer
class PeepholeLSTM(Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        super(PeepholeLSTM, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      initializer='glorot_uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                                initializer='orthogonal',
                                                name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units * 4,),
                                    initializer='zeros',
                                    name='bias')
        self.cell_bias = self.add_weight(shape=(self.units * 3,),
                                         initializer='ones',
                                         name='cell_bias')
        self.built = True

    def call(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        z = K.dot(inputs, self.kernel)
        z += K.dot(h_tm1, self.recurrent_kernel)
        z += self.bias

        z0 = z[:, :self.units]  # Input gate
        z1 = z[:, self.units:2 * self.units]  # Forget gate
        z2 = z[:, 2 * self.units:3 * self.units]  # Cell gate
        z3 = z[:, 3 * self.units:]  # Output gate

        i = K.hard_sigmoid(z0 + self.cell_bias[:self.units] * c_tm1)  # Peephole connection for input gate
        f = K.hard_sigmoid(z1 + self.cell_bias[self.units:2 * self.units] * c_tm1)  # Peephole connection for forget gate
        c = f * c_tm1 + i * K.tanh(z2)
        o = K.hard_sigmoid(z3 + self.cell_bias[2 * self.units:] * c)  # Peephole connection for output gate

        h = o * K.tanh(c)
        return h, [h, c]

# Set the path to your dataset
train_data_dir = r"D:\NEW_SMOTE\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the Peephole LSTM model using the custom PeepholeLSTM layer
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))  # Standard LSTM
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    steps_per_epoch=312,
    epochs=30)

# Save the trained model
model.save('lung_detection_standard_lstm_NOTPRE30.h5')


Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 208s 654ms/step - accuracy: 0.4717 - loss: 1.3289
Epoch 2/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 134s 429ms/step - accuracy: 0.6754 - loss: 0.8718
Epoch 3/30


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


312/312 ━━━━━━━━━━━━━━━━━━━━ 202s 646ms/step - accuracy: 0.7361 - loss: 0.7216
Epoch 4/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 125s 399ms/step - accuracy: 0.7910 - loss: 0.5788
Epoch 5/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 207s 665ms/step - accuracy: 0.8387 - loss: 0.4434
Epoch 6/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 127s 407ms/step - accuracy: 0.8446 - loss: 0.4247
Epoch 7/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 209s 670ms/step - accuracy: 0.8889 - loss: 0.3050
Epoch 8/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 128s 411ms/step - accuracy: 0.8936 - loss: 0.2997
Epoch 9/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 207s 662ms/step - accuracy: 0.9264 - loss: 0.2034
Epoch 10/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 128s 409ms/step - accuracy: 0.9263 - loss: 0.2156
Epoch 11/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 209s 670ms/step - accuracy: 0.9473 - loss: 0.1521
Epoch 12/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 131s 418ms/step - accuracy: 0.9437 - loss: 0.1647
Epoch 13/30
312/312 ━━━━━━━━━━━━━━━━━━━━ 210s 674ms/step - accuracy: 0.9680 - loss: 0.0958
Epoch 14/30
312/3

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model('lung_detection_standard_lstm_NOTPRE30.h5')

# Prepare your test dataset (similar to how you prepared the training data)
test_data_dir = r"D:\NEW_SMOTE\test-20230326T155708Z-001\test"
img_width, img_height = 150, 150
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_true)

# Make predictions on the test data
y_pred_probs = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute precision, recall, and confusion matrix
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 18s 127ms/step - accuracy: 0.7795 - loss: 1.7025
129/129 ━━━━━━━━━━━━━━━━━━━━ 18s 136ms/step
Test Loss: 1.5483472347259521
Test Accuracy: 0.7761557102203369
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.83      0.78      0.81       685
       COVID       0.78      0.67      0.72       685
      NORMAL       0.74      0.78      0.76       685
   PNEUMONIA       0.89      0.86      0.88       685
PNEUMOTHORAX       0.56      0.66      0.61       685
TUBERCULOSIS       0.90      0.89      0.90       685

    accuracy                           0.78      4110
   macro avg       0.78      0.78      0.78      4110
weighted avg       0.78      0.78      0.78      4110

Confusion Matrix:
[[535  27   0  16 100   7]
 [ 36 461  28   9 117  34]
 [ 15  24 537  25  83   1]
 [  4   9  56 592  20   4]
 [ 40  55 101  15 454  20]
 [ 11  13   7   7  36 611]]


In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

# Custom PeepholeLSTM layer
class PeepholeLSTM(Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        super(PeepholeLSTM, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      initializer='glorot_uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                                initializer='orthogonal',
                                                name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units * 4,),
                                    initializer='zeros',
                                    name='bias')
        self.cell_bias = self.add_weight(shape=(self.units * 3,),
                                         initializer='ones',
                                         name='cell_bias')
        self.built = True

    def call(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        z = K.dot(inputs, self.kernel)
        z += K.dot(h_tm1, self.recurrent_kernel)
        z += self.bias

        z0 = z[:, :self.units]  # Input gate
        z1 = z[:, self.units:2 * self.units]  # Forget gate
        z2 = z[:, 2 * self.units:3 * self.units]  # Cell gate
        z3 = z[:, 3 * self.units:]  # Output gate

        i = K.hard_sigmoid(z0 + self.cell_bias[:self.units] * c_tm1)  # Peephole connection for input gate
        f = K.hard_sigmoid(z1 + self.cell_bias[self.units:2 * self.units] * c_tm1)  # Peephole connection for forget gate
        c = f * c_tm1 + i * K.tanh(z2)
        o = K.hard_sigmoid(z3 + self.cell_bias[2 * self.units:] * c)  # Peephole connection for output gate

        h = o * K.tanh(c)
        return h, [h, c]

# Set the path to your dataset
train_data_dir = r"D:\NEW_SMOTE\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the Peephole LSTM model using the custom PeepholeLSTM layer
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))  # Standard LSTM
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    epochs=20)

# Save the trained model
model.save('lung_detection_standard_lstm20NOTPRE.h5')


Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 562s 1s/step - accuracy: 0.4997 - loss: 1.2858
Epoch 2/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 543s 1s/step - accuracy: 0.7439 - loss: 0.6930
Epoch 3/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 548s 1s/step - accuracy: 0.8305 - loss: 0.4729
Epoch 4/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 555s 1s/step - accuracy: 0.8872 - loss: 0.3072
Epoch 5/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 555s 1s/step - accuracy: 0.9239 - loss: 0.2173
Epoch 6/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 559s 1s/step - accuracy: 0.9470 - loss: 0.1522
Epoch 7/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 543s 1s/step - accuracy: 0.9642 - loss: 0.1032
Epoch 8/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 544s 1s/step - accuracy: 0.9746 - loss: 0.0750
Epoch 9/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 542s 1s/step - accuracy: 0.9770 - loss: 0.0711
Epoch 10/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 542s 1s/step - accuracy: 0.9839 - loss: 0.0492
Epoch 11/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 544s 1s/step - accuracy: 0.9890 - loss: 0.0348
Epoch 12/20
502/502 ━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model('lung_detection_standard_lstm20NOTPRE.h5')

# Prepare your test dataset (similar to how you prepared the training data)
test_data_dir = r"D:\NEW_SMOTE\test-20230326T155708Z-001\test"
img_width, img_height = 150, 150
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_true)

# Make predictions on the test data
y_pred_probs = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute precision, recall, and confusion matrix
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 30s 216ms/step - accuracy: 0.7733 - loss: 2.2516
129/129 ━━━━━━━━━━━━━━━━━━━━ 29s 221ms/step
Test Loss: 2.1385600566864014
Test Accuracy: 0.7625303864479065
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.81      0.78      0.79       685
       COVID       0.71      0.68      0.69       685
      NORMAL       0.77      0.76      0.76       685
   PNEUMONIA       0.90      0.87      0.88       685
PNEUMOTHORAX       0.56      0.63      0.59       685
TUBERCULOSIS       0.87      0.87      0.87       685

    accuracy                           0.76      4110
   macro avg       0.77      0.76      0.76      4110
weighted avg       0.77      0.76      0.76      4110

Confusion Matrix:
[[533  45   1  10  87   9]
 [ 44 463  27   8 106  37]
 [ 16  17 521  35  89   7]
 [  8  16  36 594  29   2]
 [ 45  82  83   8 430  37]
 [ 16  28  10   5  33 593]]


In [3]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

# Custom PeepholeLSTM layer
class PeepholeLSTM(Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        super(PeepholeLSTM, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                      initializer='glorot_uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                                initializer='orthogonal',
                                                name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units * 4,),
                                    initializer='zeros',
                                    name='bias')
        self.cell_bias = self.add_weight(shape=(self.units * 3,),
                                         initializer='ones',
                                         name='cell_bias')
        self.built = True

    def call(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        z = K.dot(inputs, self.kernel)
        z += K.dot(h_tm1, self.recurrent_kernel)
        z += self.bias

        z0 = z[:, :self.units]  # Input gate
        z1 = z[:, self.units:2 * self.units]  # Forget gate
        z2 = z[:, 2 * self.units:3 * self.units]  # Cell gate
        z3 = z[:, 3 * self.units:]  # Output gate

        i = K.hard_sigmoid(z0 + self.cell_bias[:self.units] * c_tm1)  # Peephole connection for input gate
        f = K.hard_sigmoid(z1 + self.cell_bias[self.units:2 * self.units] * c_tm1)  # Peephole connection for forget gate
        c = f * c_tm1 + i * K.tanh(z2)
        o = K.hard_sigmoid(z3 + self.cell_bias[2 * self.units:] * c)  # Peephole connection for output gate

        h = o * K.tanh(c)
        return h, [h, c]

# Set the path to your dataset
train_data_dir = r"D:\NEW_SMOTE\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j])
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Add a time dimension to the input data
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=1)

# Define the Peephole LSTM model using the custom PeepholeLSTM layer
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_width, img_height, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128, activation='relu', return_sequences=False))  # Standard LSTM
model.add(Dense(len(class_names), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_reshaped,
    y_train,
    batch_size=batch_size,
    steps_per_epoch=312,
    epochs=10)




Found 16050 images belonging to 6 classes.


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 340s 1s/step - accuracy: 0.4414 - loss: 1.4174
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 199s 637ms/step - accuracy: 0.6686 - loss: 0.8745
Epoch 3/10


C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


312/312 ━━━━━━━━━━━━━━━━━━━━ 328s 1s/step - accuracy: 0.7370 - loss: 0.7259
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 198s 633ms/step - accuracy: 0.7661 - loss: 0.6422
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 326s 1s/step - accuracy: 0.8177 - loss: 0.5073
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 198s 631ms/step - accuracy: 0.8245 - loss: 0.4832
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 369s 1s/step - accuracy: 0.8719 - loss: 0.3614
Epoch 8/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 201s 644ms/step - accuracy: 0.8711 - loss: 0.3440
Epoch 9/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 329s 1s/step - accuracy: 0.9125 - loss: 0.2330
Epoch 10/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 201s 642ms/step - accuracy: 0.8957 - loss: 0.2747


In [4]:
# Save the trained model
model.save('lung_detection_standard_lstm10NOTPRE.h5')

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

# Load the trained model
model = tf.keras.models.load_model('lung_detection_standard_lstm10NOTPRE.h5')

# Prepare your test dataset (similar to how you prepared the training data)
test_data_dir = r"D:\NEW_SMOTE\test-20230326T155708Z-001\test"
img_width, img_height = 150, 150
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)

# Reshape the input data to include a time dimension
X_test_reshaped = []
y_true = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j])
        y_true.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_true = np.array(y_true, dtype=np.int32)

# Add a time dimension to the input data
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=1)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_true)

# Make predictions on the test data
y_pred_probs = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute precision, recall, and confusion matrix
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Found 4110 images belonging to 6 classes.
129/129 ━━━━━━━━━━━━━━━━━━━━ 16s 121ms/step - accuracy: 0.7895 - loss: 0.7811
129/129 ━━━━━━━━━━━━━━━━━━━━ 16s 124ms/step
Test Loss: 0.8217014074325562
Test Accuracy: 0.7678832411766052
Classification Report:
              precision    recall  f1-score   support

CARDIOMEGALY       0.84      0.81      0.82       685
       COVID       0.73      0.74      0.73       685
      NORMAL       0.85      0.69      0.76       685
   PNEUMONIA       0.91      0.87      0.89       685
PNEUMOTHORAX       0.52      0.77      0.62       685
TUBERCULOSIS       0.94      0.74      0.83       685

    accuracy                           0.77      4110
   macro avg       0.80      0.77      0.78      4110
weighted avg       0.80      0.77      0.78      4110

Confusion Matrix:
[[552  52   0   2  79   0]
 [ 27 504  12   3 117  22]
 [ 12  13 474  46 136   4]
 [ 10  25  23 596  30   1]
 [ 35  69  47   4 526   4]
 [ 23  25   3   2 128 504]]
